# Bio2Py: Implementation example

Bio2Py automatices the process of loading influent data, changing parameters and running simulations in BioWin. It is necessary to set the system configuration layout, project options, unit system, report options, etc before using Bio2Py. 

In order to use Bio2Py, **BioWin simulation window must be fully visible**, as Bio2Py relies on image recognition to perform its tasks. Placing BioWin window on the left side of the screen is recomended. 

**Before using Bio2Py**:
- Open BioWin simulation file.
- Set the zoom to 100% on BioWin window.
- Manually run a single flow balance and steady state/dynamic simulation. Check for any warning messages that could interrupt Bio2Py process. If necessary, the user may consider disabling BioWin alarms to prevent Bio2Py interruption.  
- Choose locations (if applicable): 
    - For saving BioWin generated reports with simulation results (File -> Report to Excel (TM) -> Choose directory).
    - From where variable influent data will be loaded (Influent icon -> Open file -> Choose directory)(***). Only necessary if running simulations with variable influent. Choose the same filepath arg .
- Place BioWin simulation window on the left side of the screen. 
- Make sure influent icon is visible. 

**1. Loading the necessary packages and setting the environment**

In [6]:
import pandas as pd
import Bio2Py

In [7]:
simulation_window,influen_window=Bio2Py.setting_the_environment()

Filepath:  to save the .csv generated files with simulation results.

Important: change to the chosen filepath.

In [8]:
filepath=r'C:\Users\Usuario\OneDrive - Facultad de Ingeniería\Maestría\Biowin - python\short communication\Revisiones\Implementation example' #change

**2. Loading influent data**

In [9]:
column_names= ['Flow','COD (mg-COD/L)','TKN (mg-N/L)','TP (mg-P/L)','TS (mg-S/L)','Nitrate (mg-N/L)','pH','Alkalinity (mmol/L)','ISS total (mg-ISS/L)','Metal soluble - Ca (mg/L)','Metal soluble - Mg (mg/L)', 'Gas - Dissolved O2 (mg/L)'] #in the same order of appearence as in BioWin
influent= [[840.0,1040.0,32.0,36.0,10.0,0,5.1,6.0,45.0,80.0,15.0,0]] #seed value
influent= pd.DataFrame(influent, columns=column_names)
influent

,Flow,COD (mg-COD/L),TKN (mg-N/L),TP (mg-P/L),TS (mg-S/L),Nitrate (mg-N/L),pH,Alkalinity (mmol/L),ISS total (mg-ISS/L),Metal soluble - Ca (mg/L),Metal soluble - Mg (mg/L),Gas - Dissolved O2 (mg/L)
0,840.0,1040.0,32.0,36.0,10.0,0,5.1,6.0,45.0,80.0,15.0,0


**3. Running the simulations**

- Place BioWin simulation window on the left side of the screen. 
- Make sure the influent icon is visible. 

In [10]:
TP_efluent = 0
iteration = 1
it_step = 0.5
results_list=[]

while TP_efluent < 5.0:
    TP_influent = influent.iloc[0,3]
    Bio2Py.steady_state_simulations('constant',influent,'seed','current',200,'table',filepath) #Runs steady state simulation for the specified influent parameters
    efluent = pd.read_csv('steady_state_c_0.csv')
    TP_efluent = efluent.iloc[0, 6]

    print(f'Iteration {iteration}:')
    print(f'Influent TP: {TP_influent}')
    print(f'Effluent TP: {TP_efluent}')

    result_values = {'Iteration': iteration, 'Influent_TP': TP_influent}
    result_values.update(dict(zip(efluent.columns, efluent.values.flatten())))
    results_list.append(result_values)

    # Convert the list of results into a DataFrame
    results_columns = ['Iteration', 'Influent_TP'] + efluent.columns.tolist()
    results = pd.DataFrame(results_list, columns=results_columns)

    # Update influent values for the next iteration
    influent.at[0, 'TP (mg-P/L)'] = influent.at[0, 'TP (mg-P/L)'] + it_step #Influent characteristics for the next iteration (only changes TP value)
    iteration=iteration+1


21:01:42.390: Flow balance
21:01:55.511: Steady State


Iteration 1:
Influent TP: 36.0
Effluent TP: 3.93


21:02:42.994: Flow balance
21:02:55.819: Steady State


Iteration 2:
Influent TP: 36.5
Effluent TP: 4.25


21:03:43.103: Flow balance
21:03:55.956: Steady State


Iteration 3:
Influent TP: 37.0
Effluent TP: 4.59


21:04:43.493: Flow balance
21:04:56.490: Steady State


Iteration 4:
Influent TP: 37.5
Effluent TP: 4.95


21:05:43.987: Flow balance
21:05:57.171: Steady State


Iteration 5:
Influent TP: 38.0
Effluent TP: 5.32


**4. Results**

In [11]:
results

,Iteration,Influent_TP,Elements,BOD - Total Carbonaceous [mg/L],COD - Total [mg/L],N - Ammonia [mgN/L],N - Total Kjeldahl Nitrogen [mgN/L],P - Soluble PO4-P [mgP/L],P - Total P [mgP/L],pH [],Total suspended solids [mg/L]
0,1,36.0,Effluent,4.72,65.36,0.11,1.82,3.43,3.93,6.71,7.53
1,2,36.5,Effluent,4.72,65.36,0.11,1.82,3.75,4.25,6.70,7.54
2,3,37.0,Effluent,4.72,65.36,0.11,1.82,4.09,4.59,6.70,7.55
3,4,37.5,Effluent,4.72,65.36,0.11,1.82,4.44,4.95,6.70,7.55
4,5,38.0,Effluent,4.72,65.36,0.10,1.82,4.81,5.32,6.70,7.56
